# Automatic Deep Research 

Welcome to this new practice lab! By now you should have a clearer view of the elements that compose a multi-agent system. In this lab you will get to put it into action by creating your first crew.

**What you'll learn:**
- How to define agents with specific roles and expertise
- How to provide agents with tools to perform their tasks
- How to create your own tasks that agents will execute
- How to assemble agents and tasks into a Crew, all using CrewAI

## Background

As a research consultant, you're constantly tasked with producing comprehensive reports on diverse topics for demanding clients. You need to build an automatic deep research solution that can rapidly gather, verify, and synthesize information from across the internet, delivering reliable, fact-checked reports that meet tight deadlines and exacting standards regardless of the subject matter. 

## General instructions
In this lab you will be presented with a structure of the code, but you will need to complete some of it. 

To successfully run this lab, replace all instances of the placeholder `None` with your own code. Sections where you need to write code will be delimited between `### START CODE HERE ###` and `### END CODE HERE ###`.

If you are stuck, or simply want to copy a solution into your notebook so that you can execute it, you can find all solution code inside the [Solution](Solution) folder.

**<font color='#5DADEC'>Please make sure to save your work periodically, so you don't lose any progress.</font>**

## Table of contents

- [1. Understanding the problem](#1)
- [2. Set up your notebook](#2)
- [3. Define the Agents](#3)
  - [3.1. Create tool instances](#3-1)
  - [3.2. Define the Research Planner agent](#3-2)
  - [3.3. Define the remaining agents](#3-3)
- [4. Create the Tasks](#4)
  - [4.1. Define the Create research plan task](#4-1)
  - [4.2. Define the remaining tasks](#4-2)
- [5. Define the Crew and get the results](#5)

<a id="1"></a>

## 1. Understanding the problem
In this lab, you will focus on building a custom deep research crew. This Crew will be in charge of creating a research plan based on the user's input, and executing it, while reviewing and checking the facts. Finally, with the gathered information a report needs to be generated.

Take some time to decompose the problem into different tasks. Who would be the appropriate "person" to solve each task? 

Once you've done your thinking, click below to find an agent/task diagram for this lab.    


<details>    
<summary>
    <font size="3" color="#237b946b"><b>Diagram</b></font>
</summary>

<img src="../images/lab2-agents-tasks-diagram.PNG">

<a id="2"></a>

## 2. Set up your notebook

Before you start coding, run the next two cells to import all necessary modules and configure the environment variables. 

In [4]:
from crewai import Agent, Task, Crew
import os
os.environ["CREWAI_TESTING"] = "true"
from utils import get_openai_api_key

# set the OpenAI model (gpt-4o-mini)
os.environ["MODEL"] = "gpt-4o-mini"
# set up the OpenAI API key 
os.environ["OPENAI_API_KEY"] = get_openai_api_key()

<a id="3"></a>

## 3. Define the Agents

Based on the diagram, you should have four agents:
- **Research Planner**: its goal is to analyze queries and break them down into smaller, specific research topics.
- **Internet Researcher**: its job is to perform research tasks.
- **Fact checker**: its goal is to review information for fact accuracy to avoid misinformation. 
- **Report Writer**: is in charge of writing reports, based on gathered information.

<a id="3-1"></a>

### 3.1. Create tool instances
As you can see in the diagram, you will be providing the **Internet Researcher Agent** with tools, so that it can better do their job. In particular, you will give this agent access to search the internet and scrape information from the retrieved webpages. 

There are different tools inside CrewAI you can use to search the web, in this lab you will use the [**EXA Search Web Loader**](https://docs.crewai.com/en/tools/search-research/exasearchtool#exa-search-web-loader) tool, which is designed to perform a semantic search for a specified query from a text’s content across the internet. It utilizes the [exa.ai](https://exa.ai/) API to fetch and display the most relevant search results based on the query provided by the user. exa.ai enhances semantic search by capturing richer contextual relationships between concepts, allowing for more precise information retrieval than conventional embedding approaches.

For webscraping, you will use the [**Scrape Website**](https://docs.crewai.com/en/tools/web-scraping/scrapewebsitetool) tool, which is designed to extract and read the content of a specified website.

In the next cell you will define instances of these tools, so you can later assign them to the agents.

In [5]:
# import the tools
from crewai_tools import EXASearchTool, ScrapeWebsiteTool
from utils import get_exa_api_key

# set the exa API key
os.environ["EXA_API_KEY"] = get_exa_api_key()

### START CODE HERE ###

# Create the EXASearchTool instance
exa_search_tool = EXASearchTool(base_url=os.getenv("EXA_BASE_URL"))
# Create the ScrapeWebsiteTool instance
scrape_website_tool = ScrapeWebsiteTool()

### END CODE HERE ###

<a id="3-2"></a>

### 3.2. Define the Research Planner agent

In the cell below, you will see how you can create the first agent. This time, all the parameters are set up for you. Here is a quick recap of what each of the parameters represent:

- `Role`: If this was a person doing the job, what title would they have?
- `Goal`: What is the goal this agent in particular is trying to accomplish? Make sure to write concrete goal
- `Background`: it should be something the highlights the skills of the agent relevant to its role. Make sure to use keywords that will actually help your agent get better results.

In the labs, we have added two parameters not shown in the demo videos: `max_rpm`, and `max_iter`. `max_rpm` sets the maximum requests per minute to avoid rate limits, while `max_iter` limits the maximum iterations before the agent must provide its best answer. Setting these two parameters helps make the agents run a little faster, so the lab doesn't take as long to complete. 

In [6]:
# define the research planner agent
research_planner = Agent(
    role="Research Planner",
    goal="Analyze queries and break them down into smaller, specific research topics.",
    backstory=(
         "You are a research strategist who excels at breaking down complex questions "
         "into manageable research components. You identify what needs to be researched "
         "and create clear research objectives."
    ),
    verbose=True, # set to True to see detailed agent actions
    max_rpm=150,
    max_iter=15
)

<a id="3-3"></a>

### 3.3. Define the remaining agents

Now you can define the three remaining agents. The `role` and `goal` parameters are already filled in for you; use your own creativity to fill in the `backstory`.  

Do not forget to assign the tools to the **Internet Researcher** and **Fact Checker** agents. You can do this by setting the `tools` argument.

In [7]:

researcher = Agent(
    role="Internet Researcher",
    goal="Research thoroughly all assigned topics",
    ### START CODE HERE ###
    backstory=(
        "You are a skilled researcher with experience in online investigation "
        "and data collection. You know how to find reliable sources, extract relevant information, "
        "and always verify facts across multiple sources to avoid misinformation or hallucination. "
        "You never invent facts and always trace information to its origin."
    ),
    # add the 2 tool instances you created
    tools=[exa_search_tool, scrape_website_tool],
    ### END CODE HERE ###
    verbose=True,
    max_rpm=150,
    max_iter=15
)

fact_checker = Agent(
    role="Fact Checker",
    goal=(
        "Verify data for accuracy, identify inconsistencies, "
        "and flag potential misinformation"
    ),
    ### START CODE HERE ###
    backstory=(
        "You are a quality assurance specialist with expertise in fact-checking "
        "and identifying misinformation and hallucinations. You cross-reference information, "
        "spot inconsistencies, and ensure all data meets high accuracy standards. You rigorously"
        "check for hallucinated or invented content and require that all facts be supported"
        "by evidence."
    ),
    tools=[exa_search_tool, scrape_website_tool],
    ### END CODE HERE ###
    verbose=True,
    max_rpm=150,
    max_iter=15
)

report_writer = Agent(
    role="Report Writer",
    goal="Write clear, concise, and well-structured reports based on gathered information",
    ### START CODE HERE ###
    backstory=(
         "You are an expert writer who specializes in creating clear, well-structured "
        "research reports. You synthesize complex information into readable formats and "
        "always include proper citations and sources."
    ),
    ### END CODE HERE ###
    verbose=True,max_rpm=150,
    max_iter=15
)
          

<a id="4"></a>

## 4. Create the Tasks

Now that you have set up the agents, it is time to define the tasks. If you go back to the diagram, you will see you need four tasks:

- **Create research plan**: Based on the user's query, break it down into specific topics and key questions, and create a focused research plan.
    - Output: A research plan with main research topics to investigate, key questions for each topic, and success criteria for the research.

- **Gather research data**: Using the research plan, collect information on all identified topics. Cite all sources used.
    - Output: Comprehensive research data including: information for each research topic, and citations used along with source credibility notes.

- **Verify information quality**: Review all collected research. Identify any conflicting information, potential misinformation, or gaps that need addressing.
    - Output: A report with the all the collected data, and its review. It should include consistency check results and source reliability ratings

- **Write final report**: Create a comprehensive report that answers the original query using all verified research data. Structure it with clear sections, include citations, and provide actionable insights.
    - Output: The final research report. In addition to the full answer, it should have an executive summary, and complete source citations.


For each `Task` you need to define the following parameters:
- `description`: A thorough description of the task. You can even break it down into different items.
- `expected_output`: what should the output return. Be specific, specially if you want any structure in your result, like a dictionary with specific keys.
- `agent`: who is performing the task? You need to match the task to one of the agents you already defined

In the description you will need to pass the inputs to the tasks. In this lab, you will only have as input the user's query, which will be saved as `user_query`:


<a id="4-1"></a>

### 4.1. Define the Create research plan task

In the cell below, you will see how you can create the first task. This time, all the parameters are set up for you. Notice how the context variables are passed the the description between curly brackets. 

In [8]:
# define the create research plan task
create_research_plan_task = Task(
    description=(
        "Based on the user's query, break it down into specific topics and key questions, "
        "and create a focused research plan."
        "The user's query is: {user_query}"
    ),
    expected_output=(
        "A research plan with main research topics to investigate, "
        "key questions for each topic, and success criteria for the research."
        ),
    agent=research_planner,
)

<a id="4-2"></a>

### 4.2. Define the remaining tasks

Now define the three remaining tasks. The `description` is already filled in for you, you will need to define the `expected_output` and `agent` for each of the Tasks.

In [9]:
# define the gather research data task
gather_research_data_task = Task(
    description=(
        "Using the research plan, collect information on all identified topics. "
        "Cite all sources used."
    ),
    ### START CODE HERE ###
    expected_output=(
        "Comprehensive research data including: information for each"
        "research topic, and citations used along with source credibility notes"
    ),
    agent=researcher,
    ### END CODE HERE ###
)

#define the verify information quality task
verify_information_quality_task = Task(
    description=(
        "Review all collected research. Identify any conflicting information, "
        "potential misinformation, or gaps that need addressing."
    ),
    ### START CODE HERE ###
    expected_output=(
        "A report with all the original data you got plus any "
        "verified facts vs. questionable information, make sure this is as comprehensive "
        "as possible for final report generation"
    ),
    agent=fact_checker,
    ### END CODE HERE ###
)

# define the write final report task
write_final_report_task = Task(
    description=(
        "Create a comprehensive report that answers the original query using all verified research data. "
        "Structure it with clear sections, include citations, and provide actionable insights."
    ),
    ### START CODE HERE ###
    expected_output=(
        "A final research report containing: executive summary, detailed "
        "findings that answer the user query, supporting evidence and analysis, complete "
        "source citations."
    ),
    agent=report_writer,
    ### END CODE HERE ###
)
    

<a id="5"></a>

## 5. Define the Crew and get the results

Once the agents and tasks have been defined, you are ready to create the crew. In order to so, you will need to set the following arguments:
- `agents`: list of agents in the crew
- `tasks`: list of tasks in the crew. The tasks should be listed in the order they should be executed

In the next cell, fill in the agents and tasks for the crew.

In [11]:
# create the crew with the defined agents and tasks
crew = Crew(
    ### START CODE HERE ###
    agents=[research_planner, researcher, fact_checker, report_writer],
    tasks=[create_research_plan_task, gather_research_data_task, verify_information_quality_task, write_final_report_task],
    ### 
)

Before running the crew, you need to define the query, which will be used as input for the tasks.

In [12]:
### START CODE HERE ###

# Write your query, which will be used as input for the tasks.
user_query = "Evaluate the top five emerging AI tools for automating competitive market analysis, including their features, limitations, costs, and ideal use cases for a mid-sized marketing firm."

### END CODE HERE ###

Now you are only left with kickstarting the crew to get the results. Since you set `verbose=True` in the agents, you should monitor all the process.

In [13]:
result = crew.kickoff(
    inputs={
        "user_query": user_query,
    }
)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Planner                                                                                        │
│                                                                                                                 │
│  Task: Based on the user's query, break it down into specific topics and key questions, and create a focused    │
│  research plan.The user's query is: Evaluate the top five emerging AI tools for automating competitive market   │
│  analysis, including their features, limitations, costs, and ideal use cases for a mid-sized marketing firm.    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Planner                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Research Plan for Evaluating Top Five Emerging AI Tools for Automating Competitive Market Analysis**         │
│                                                                                                                 │
│  **Main Research Topics:**                                                                                      │
│                                                                                                                 │
│  1. **Identification of AI Tools**                                                                              │
│     - Key Questions:                                                                                            │
│       - What are the current top five AI tools specifically designed for automating competitive market          │
│  analysis?                                                                                                      │
│       - How are these tools categorized (e.g., based on pricing, features, functionality)?                      │
│     - Success Criteria:                                                                                         │
│       - A comprehensive list of at least five reputable AI tools with a brief overview of each, including its   │
│  market presence and recognition.                                                                               │
│                                                                                                                 │
│  2. **Features of Each Tool**                                                                                   │
│     - Key Questions:                                                                                            │
│       - What are the key features of each identified AI tool?                                                   │
│       - How do these features support competitive market analysis specifically?                                 │
│       - Are there any user-friendly aspects of the tools that make them suitable for mid-sized marketing        │
│  firms?                                                                                                         │
│     - Success Criteria:                                                                                         │
│       - Detailed breakdown of features for each tool, including a comparison chart highlighting strengths and   │
│  unique capabilities.                                                                                           │
│                                                                                                                 │
│  3. **Limitations and Challenges**                                                                              │
│     - Key Questions:                                                                                            │
│       - What limitations and challenges are associated with each AI tool?                                       │
│       - Are there specific use scenarios where these tools might underperform?                                  │
│       - What feedback do users provide regarding downsides or difficulties?                                     │
│     - Success Criteria:                                                                                         │
│       - A thorough analysis of constraints for each too

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Researcher                                                                                     │
│                                                                                                                 │
│  Task: Using the research plan, collect information on all identified topics. Cite all sources used.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Researcher                                                                                     │
│                                                                                                                 │
│  Thought: I must gather comprehensive information on the top five AI tools for automating competitive market    │
│  analysis according to the research plan. This requires searching for current tools, their features,            │
│  limitations, costs, and ideal use cases.                                                                       │
│                                                                                                                 │
│  Using Tool: EXASearchTool                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "top AI tools for automating competitive market analysis 2023",                              │
│    "start_published_date": null,                                                                                │
│    "end_published_date": null,                                                                                  │
│    "include_domains": null                                                                                      │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "website_url": "https://visualping.io/blog/best-ai-tools-competitor-analysis"                                │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The following text is scraped website content:                                                                 │
│                                                                                                                 │
│  10 Best AI Tools for Competitor Analysis in 2025: Stay Ahead with Automated Intelligence Product Technology    │
│  How it works Web change monitoring, explained Visualping AI Let AI do the work API Integrations Integrate      │
│  into your workflow Products Chrome extension Monitor in a click Button Keep your site visitors in the know     │
│  Public pages Manage your Visualping alert page Customers Personal Use Back in stock Know when to act fast      │
│  with restock alerts Price drops Shop smarter and save with the latest deals Appointment availability Get       │
│  notified as soon as there's an open slot News Stay on top of the latest headlines Job hunting Monitor          │
│  openings at your dream company House hunting Track new listings that match your criteria Social media Monitor  │
│  profile updates and new posts Grades and courses Get alerted of grades and registration dates Restaurant       │
│  reservations Score a table at your favourite restaurant Scholarships Keep an eye on scholarship updates        │
│  Travel planning Snag hard-to-get hotels and flights Tickets and events Know the moment you can score a ticket  │
│  Business Use cases Competitor monitoring Stay on top of your competitors Regulatory intelligence Get           │
│  real-time updates straight from the source Compliance monitoring Catch non-compliance anywhere online          │
│  Regression & defacement monitoring Catch irregularities before your customers do SEO monitoring Detect issues  │
│  before your rankings drop Industry solutions Investment management Be the first to know about market-moving    │
│  events Law Firms Know every legal update before your clients do SaaS & information services Keep your content  │
│  up-to-date with web change data eCommerce Stay competitive by tracking price and inventory changes Insurance   │
│  Detect underwriting risk across your client base Journalism Detect the updates you need to source your best    │
│  story yet Pricing Personal use Individuals, students and families Teams & small business Teams & small         │
│  business Enterprise solutions A ful...                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Fact Checker                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Comprehensive Evaluation of Top Five Emerging AI Tools for Automating Competitive Market Analysis**          │
│                                                                                                                 │
│  ### 1. Madgicx AI Marketer                                                                                     │
│  - **Overview**: Madgicx provides powerful automated marketing solutions that focus on competitor insights and  │
│  campaign optimization.                                                                                         │
│  - **Features**:                                                                                                │
│    - Automated monitoring of competitor ads across platforms.                                                   │
│    - AI-driven insights and integrated campaign optimization.                                                   │
│    - Competitive data analytics for better ad performance.                                                      │
│  - **Limitations**: Primarily focused on digital advertising data.                                              │
│  - **Cost**: Starts at $99/month.                                                                               │
│  - **Ideal Use Cases**: E-commerce businesses and agencies aiming to enhance advertisement efficiencies.        │
│                                                                                                                 │
│  ### 2. Visualping                                                                                              │
│  - **Overview**: Visualping excels at automating website change monitoring and competitor updates.              │
│  - **Features**:                                                                                                │
│    - AI-based detection of significant website changes.                                                         │
│    - Custom alerts and detailed change summaries.                                                               │
│  - **Limitations**: Focus is mainly on web content changes, lacking broader competitive metrics.                │
│  - **Cost**: Plans start at $13/month.                                                                          │
│  - **Ideal Use Cases**: Professionals in competitive intelligence and product management who require timely     │
│  site updates.                                                                                                  │
│                                                                                                                 │
│  ### 3. SEMRush                                                                                                 │
│  - **Overview**: SEMRush is a comprehensive tool known for SEO and competitive market intelligence.             │
│  - **Features**:                                                                                                │
│    - Detailed analysis of organic and paid search.                                                              │
│    - Tools for identifying keyword opportunities.                                                               │
│    - In-depth backlink profiles monitoring.                                                                     │
│  - **Limitations**: May present a steep learning curve.

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Task: Create a comprehensive report that answers the original query using all verified research data.          │
│  Structure it with clear sections, include citations, and provide actionable insights.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Comprehensive Evaluation of Top Five Emerging AI Tools for Automating Competitive Market Analysis          │
│                                                                                                                 │
│  #### Executive Summary                                                                                         │
│  This report evaluates the top five emerging AI tools for automating competitive market analysis, focusing on   │
│  their features, limitations, costs, and ideal use cases tailored for mid-sized marketing firms. The aim is to  │
│  provide actionable insights to facilitate informed decision-making in adopting these technologies for more     │
│  effective competitive analysis and strategy formulation.                                                       │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 1. Madgicx AI Marketer                                                                                     │
│  - **Overview**: Madgicx delivers advanced tools for competitive intelligence by automating the monitoring of   │
│  competitors' advertising strategies. It optimizes users' ad campaigns based on real-time competitor            │
│  performance data and insights.                                                                                 │
│  - **Features**:                                                                                                │
│    - Automated monitoring of competitor ads across platforms (Facebook, Google, etc.).                          │
│    - Real-time insights into competitors’ advertising strategies and performance metrics.                       │
│    - Integrated campaign optimization features informed by competitive data.                                    │
│  - **Limitations**: The tool primarily focuses on digital advertising data and may not encompass other aspects  │
│  of competitive analysis, such as product benchmarking or pricing intelligence.                                 │
│  - **Cost**: Pricing structures begin at $99/month, varying based on the ad spend, with additional tiers for    │
│  more comprehensive features.                                                                                   │
│  - **Ideal Use Cases**: Particularly beneficial for e-commerce businesses and digital marketing agencies keen   │
│  on maximizing return on ad spend and optimizing campaigns based on competitor benchmarking.                    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 2. Visualping                                                                                              │
│  - **Overview**: Visualping specializes in real-time monitoring of website changes, allowing users to track     │
│  updates on competitor sites effectively.              

From the output of the previous cell check all the outputs for each task. Do they match what you expected? If not, go back and refine the `expected_output`. 

You can also print the final report to see the final result of the crew

In [14]:
from IPython.display import Markdown
Markdown(result.raw) 

### Comprehensive Evaluation of Top Five Emerging AI Tools for Automating Competitive Market Analysis

#### Executive Summary
This report evaluates the top five emerging AI tools for automating competitive market analysis, focusing on their features, limitations, costs, and ideal use cases tailored for mid-sized marketing firms. The aim is to provide actionable insights to facilitate informed decision-making in adopting these technologies for more effective competitive analysis and strategy formulation.

---

### 1. Madgicx AI Marketer
- **Overview**: Madgicx delivers advanced tools for competitive intelligence by automating the monitoring of competitors' advertising strategies. It optimizes users' ad campaigns based on real-time competitor performance data and insights.
- **Features**:
  - Automated monitoring of competitor ads across platforms (Facebook, Google, etc.).
  - Real-time insights into competitors’ advertising strategies and performance metrics.
  - Integrated campaign optimization features informed by competitive data.
- **Limitations**: The tool primarily focuses on digital advertising data and may not encompass other aspects of competitive analysis, such as product benchmarking or pricing intelligence.
- **Cost**: Pricing structures begin at $99/month, varying based on the ad spend, with additional tiers for more comprehensive features.
- **Ideal Use Cases**: Particularly beneficial for e-commerce businesses and digital marketing agencies keen on maximizing return on ad spend and optimizing campaigns based on competitor benchmarking.
  
---

### 2. Visualping
- **Overview**: Visualping specializes in real-time monitoring of website changes, allowing users to track updates on competitor sites effectively.
- **Features**:
  - AI-powered detection of changes on competitor websites (new content, design updates, etc.).
  - Customizable alerts for significant changes detected.
  - Comprehensive summaries of changes for detailed competitor analysis.
- **Limitations**: The focus is strictly on web content changes, which may omit broader competitive metrics like audience insights or market share analytics.
- **Cost**: Basic plans start at $13/month, making it accessible for smaller firms needing essential monitoring capabilities.
- **Ideal Use Cases**: Ideal for competitive intelligence professionals and product managers who require ongoing updates about competitors’ offerings, promotional strategies, or changes in product lines.

---

### 3. SEMRush
- **Overview**: SEMRush is a powerful digital marketing SaaS tool providing extensive analytics on SEO, PPC, keyword strategies, and competitive intelligence.
- **Features**:
  - In-depth competitive research for both organic and paid search.
  - Tools for identifying keyword opportunities and analyzing backlink profiles.
  - SEM audit tools to optimize user websites and improve their search rankings.
- **Limitations**: Potentially time-consuming setup and a steep learning curve; it also may require additional training to leverage the full suite effectively.
- **Cost**: Subscription ranges from $117.33 to $499/month, based on the features and capabilities needed.
- **Ideal Use Cases**: Best suited for businesses aiming to enhance their SEO performance and gain comprehensive insights into competitor keyword strategies and digital marketing approaches.

---

### 4. Similarweb
- **Overview**: Similarweb offers robust digital market intelligence, providing insights on traffic sources, user engagement, and competitors' online presence.
- **Features**:
  - Comprehensive traffic analytics encompassing websites and app performance.
  - Insights into market share and effectiveness of various traffic channels.
  - Tools for conducting competitor benchmarking across multiple dimensions.
- **Limitations**: Higher pricing tiers provide in-depth data, but page-level insights may be limited compared to dedicated SEO tools.
- **Cost**: Pricing starting at $199/month, reflecting its premium positioning in the market intelligence space.
- **Ideal Use Cases**: Fits businesses needing high-level overviews of competitors’ online performance and market positioning to derive strategic insights efficiently.

---

### 5. SpyFu
- **Overview**: SpyFu specializes in tracking and analyzing competitors’ search marketing strategies, offering insights into their keyword and PPC performance.
- **Features**:
  - Comprehensive tracking of competitor keyword usage and SERP positions.
  - Historical data on competitors’ marketing strategies over time, valuable for trend analysis.
  - Reports designed for ease of use that help marketers understand search behavior patterns.
- **Limitations**: Primarily focused on search marketing, potentially neglecting wider market intelligence aspects such as social media strategies or price comparisons.
- **Cost**: Starting plans are as low as $39/month, providing a budget option for firms focused on search marketing analytics.
- **Ideal Use Cases**: Best used by businesses that need to understand competitors’ SEO and PPC dynamics, facilitating informed strategic decisions in digital marketing.

---

### Summary of Key Features and Limitations

| Tool      | Key Features                                                  | Limitations                                    | Starting Cost          |
|-----------|--------------------------------------------------------------|------------------------------------------------|------------------------|
| Madgicx   | Automated ad monitoring, real-time insights, integrated optimization | Primarily focused on advertising data          | $99/month              |
| Visualping | AI website change detection, customizable alerts            | Narrow focus on web content changes            | $13/month              |
| SEMRush   | SEO and PPC analytics, keyword tracking                      | Time-consuming setup                           | $117.33/month          |
| Similarweb| Comprehensive traffic analysis, market share insights        | Expensive; limited page-level detail           | $199/month             |
| SpyFu     | SERP tracking, keyword performance insights                   | Limited to search marketing                     | $39/month              |

---

### Conclusion
The evaluation of these five AI tools reveals diverse functionalities tailored towards enhancing competitive market analysis for mid-sized marketing firms. By understanding each tool's features, limitations, costs, and ideal use cases, firms can make strategic investments that align with their specific needs and budget constraints. Key industry trends and competitive dynamics will dictate the choices firms must make to harness these tools effectively for maximizing holistic market insights and achieving superior positioning in an increasingly digital landscape.

#### References
1. Madgicx. (2023). 15 Best AI Tools for Competitive Benchmarking (2025 Guide). Retrieved from [Madgicx](https://madgicx.com/blog/competitive-benchmarking-ai).
2. Visualping. (2023). 10 Best AI Tools for Competitor Analysis in 2025. Retrieved from [Visualping](https://visualping.io/blog/best-ai-tools-competitor-analysis).
3. The Digital Elevator. (2023). 8 Best Competitive Intelligence Tools in 2025. Retrieved from [The Digital Elevator](https://thedigitalelevator.com/blog/best-competitive-intelligence-tools).

You made it to the end of the lab! You can go back and experiment with the goals and backstories of the agents, as well as description and expected outputs of tasks. You can also change the inputs to any research topic you wish. Have fun with it!